## Bibliotecas

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
import itertools

## Funções

In [23]:
# Carregar dados
def carregar_dados(moeda1, moeda2):
    caminho_base = '../../data/fechamentos'
    path_ativo_1 = f'../../data/fechamentos/{moeda1}USDT_5m_data.csv'
    path_ativo_2 = f'../../data/fechamentos/{moeda2}USDT_5m_data.csv'



    ativo1 = pd.read_csv(path_ativo_1, parse_dates=['timestamp'], index_col='timestamp')
    ativo2 = pd.read_csv(path_ativo_2, parse_dates=['timestamp'], index_col='timestamp')
    df = pd.DataFrame({
        'Ativo1': ativo1['close'],
        'Ativo2': ativo2['close']
    }).dropna()
    return df



# Teste de cointegração
def testar_cointegracao(series1, series2):
    _, pvalor, _ = coint(series1, series2)
    return pvalor

# Zscore
def calcular_zscore(spread, window=60):
    rolling_mean = spread.rolling(window=window).mean() #média da última janela
    rolling_std = spread.rolling(window=window).std() #desvio padrão da última janela
    return (spread - rolling_mean) / rolling_std #zscore = (x- média_janela)/(desvio_padrao_janela)


# Estratégia de sinalização
def gerar_sinais(df, zscore_compra_e_venda, zscore_encerrar_posicao, stop_loss, cooldown_stop_loss, janela):
    series1 = df['Ativo1']
    series2 = df['Ativo2']
    spread = series1 - series2
    zscore = calcular_zscore(spread, window=janela) #Zscore com janela movel
    janela=60

    nomes_posicoes = ["neutro", "compra_1_vende_2", "vende_1_compra_2"]
    posicao = nomes_posicoes[0] #comprado_vendido, vendido_comprado, encerrar, Neutro

    
    sinais_compra_e_venda = []

    for i in range(len(df)):
        linha_df = df.iloc[i]
        if zscore.iloc[i] > 1:
            posicao = nomes_posicoes[2] #entra vendido no ativo 1 e comprado no 2
        if zscore.iloc[i] < -1:
            posicao = nomes_posicoes[1] #entra comprado no ativo 1 e vendido no 2
        if (-0.5 < zscore.iloc[i] < 0.5):
            posicao = nomes_posicoes[0] #posicao mantem neutra ou encerra posicao anterior
        ##if stoploss
        sinais_compra_e_venda.append(posicao)
    
    # 📄 Resultado final
    df_sinais = pd.DataFrame(sinais_compra_e_venda)
    df_sinais.to_csv('estrategia.csv', index=False)

    return df_sinais
    

# Teste da estratégia

In [ ]:

# Algoritmo de otimizacao
zscv = 1
zse = 0.5
stop_loss = 0
cooldown = 0
janela = 60



dfcodigo = carregar_dados("ETH", "XRP")
gerar_sinais(dfcodigo, zscv, zse, stop_loss, cooldown, janela)

,0
0,neutro
1,neutro
2,neutro
3,neutro
4,neutro
...,...
210234,compra_1_vende_2
210235,compra_1_vende_2
210236,compra_1_vende_2
210237,compra_1_vende_2


## Testar pares de cointegrados

In [9]:

moedas = ['BTC', 'WBTC', 'ETH', 'ADA', 'BNB', 'SOL', 'XRP']
caminho_base = '../../data/fechamentos'

path_ativo_1 = f'../../data/fechamentos/{moedas[1]}USDT_5m_data.csv'
path_ativo_2 = f'../../data/fechamentos/{moedas[2]}USDT_5m_data.csv'

stop_loss_pct = -0.02
cooldown_periodos = 12

# Carregar todos os dados
df = carregar_dados (path_ativo_1, path_ativo_2)

# Intervalo para teste de cointegração
data_inicial = '2025-06-20'
data_final   = '2025-07-19'
df_filtrado = df.loc[data_inicial:data_final]

pvalor = testar_cointegracao(df_filtrado['Ativo1'], df_filtrado['Ativo2'])
print(f'P-valor da cointegração no período {data_inicial} a {data_final}: {pvalor:.4f}')


def carregar_ativo(nome):
    caminho = f'{caminho_base}/{nome}USDT_5m_data.csv'
    df = pd.read_csv(caminho, parse_dates=['timestamp'], index_col='timestamp')
    return df['close'].loc[data_inicial:data_final]


# 📊 Rodar teste de cointegração em todos os pares
resultados = []

for moeda1, moeda2 in itertools.combinations(moedas, 2):
    try:
        serie1 = carregar_ativo(moeda1)
        serie2 = carregar_ativo(moeda2)

        df = pd.DataFrame({moeda1: serie1, moeda2: serie2}).dropna()

        if len(df) < 100:
            continue  # ignora séries com dados insuficientes

        pvalor = coint(df[moeda1], df[moeda2])[1]

        resultados.append({
            'Ativo1': moeda1,
            'Ativo2': moeda2,
            'P-valor': round(pvalor, 5),
            'N_observacoes': len(df)
        })

    except Exception as e:
        print(f'Erro ao processar {moeda1} x {moeda2}: {e}')

# 📄 Resultado final
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values('P-valor')
df_resultados.to_csv('pares_cointegrados.csv', index=False)

# 📋 Mostrar apenas os pares cointegrados
cointegrados = df_resultados[df_resultados['P-valor'] < 0.05]
print(f"\nPares cointegrados encontrados (p < 0.05):\n")
print(cointegrados)


P-valor da cointegração no período 2025-06-20 a 2025-07-19: 0.8594

Pares cointegrados encontrados (p < 0.05):

   Ativo1 Ativo2  P-valor  N_observacoes
0     BTC   WBTC  0.00000           8535
18    BNB    SOL  0.00082           8537
14    ETH    XRP  0.02197           8536
12    ETH    BNB  0.02754           8536
17    ADA    XRP  0.04803           8537
